In [ ]:
import xarray as xr
import pandas as pd

In [ ]:
from glob import glob

In [ ]:
gt_path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/'
#irrigation # rainfall 
path_brandenburg = gt_path + 'irrigation+_archive/In situ data/German/Irrigation_Test_Site_Brandenburg.nc'
path_niedersachsen = gt_path + 'irrigation+_archive\Input netcdf\TEST_SITE_GERMANY_NIEDERSACHSEN.nc'
path_budrio = gt_path + 'irrigation+_archive/In situ data/Italy/Irrigation_Test_Site_Budrio.nc'

# Tereno
path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/TERENO_archieve'
files_BF = glob(path + '/*BF*')

In [ ]:
#Tereno

test = pd.read_csv('C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/TERENO_archieve/altPlestlin_BF1_v1.2_20121029_20210630/altPlestlin_BF1_v1.2_20121029_20210630.csv')
test.columns

In [ ]:
brandenburg = xr.load_dataset(path_niedersachsen)
#field_1 = brandenburg[['Longitude_1', 'Latitude_1', 'Time-days_1']]
brandenburg

In [ ]:
field_1